In [ ]:
#TREE INTERPRETER

from treeinterpreter import treeinterpreter as ti
instances = X_test.values
prediction, bias, contributions = ti.predict(model, instances)

print("prediction",prediction[0])
print("bias",bias[0])
print("Contributions", contributions[0])

# Convert predictions to a DataFrame for easier filtering
predictions_df = pd.DataFrame(prediction, columns=['class_0', 'class_1'])

# Filter to get only the instances predicted as class 1
predicted_ones = predictions_df[predictions_df['class_1'] > 0.5].index

# Now, you can loop over these instances and print out their contributions
for i in predicted_ones:
    print(f"Instance {i}:")
    print("Prediction:", prediction[i])
    print("Bias:", bias[i])
    print("Contributions:", contributions[i])

#Extract the number of classes
n_classes = contributions.shape[2]

#reshape bias list
if bias.ndim == 2 and bias.shape[1] == 2:
    bias = bias[:,1]

contributions_df = pd.DataFrame(contributions[:,:,1], columns=X.columns)
contributions_df['bias'] = bias
contributions_df['prediction'] = prediction[:,1]
contributions_df['true_value'] = y_test.values

contributions_df.shape

#Checking Individual Instances for their true value, predicted probability, bias
instance_index = 1
print(f"True Value: {y_test.iloc[instance_index]}")
print(f"Predicted Probability:{prediction[instance_index,1]}")
print(f"Bias (average prediction):{bias[instance_index]}")
print(f"Feature contributions: \n{contributions_df.iloc[instance_index][X.columns]}")

import matplotlib.pyplot as plt

contribs =  contributions_df.iloc[instance_index][X.columns]

#Get top 10 positive and top 10 negative contributions
top_positive_contribs = contribs.nlargest(10)
top_negative_contribs = contribs.nsmallest(10)

top_contribs = pd.concat([top_positive_contribs, top_negative_contribs])

plt.figure(figsize=(10,6))
top_contribs.plot(kind='barh', color=['green' if v > 0 else 'red' for v in top_contribs])
plt.title(f'Top 10 Positive and Negative Feature Contributions for Instance {instance_index}')
plt.xlabel('Feature')
plt.ylabel('Contribution')
plt.show()

all_instances_data = []

# Loop through the first five instances
for instance_index in range(146452):
    # Create a dictionary for the current instance
    print("'True Value': {y_test.iloc[instance_index]}")
    print("'Predicted Probability': {prediction[instance_index, 1]}")
    print("'Bias (average prediction)': {bias[instance_index]}")
    

# Create an empty list to collect the data
all_instances_data = []

# Loop through the first five instances
for instance_index in range(146452):
    # Create a dictionary for the current instance
    instance_data = {
        'True Value': y_test.iloc[instance_index],
        'Predicted Probability': prediction[instance_index, 1],
        'Bias (average prediction)': bias[instance_index]
    }
    
    # Add the feature contributions and actual values for the current instance to the dictionary
    for feature_index, feature_name in enumerate(X_test.columns):
        instance_data[f'{feature_name} Contribution'] = contributions[instance_index, feature_index, 1]
        instance_data[f'{feature_name} Value'] = X_test.iloc[instance_index, feature_index]
    
    # Append the instance data to the list
    all_instances_data.append(instance_data)

# Convert the list of dictionaries to a DataFrame
all_instances_df = pd.DataFrame(all_instances_data)

# Export the DataFrame to an Excel file
all_instances_df.to_excel('all_instances_contributions.xlsx', index=False)

new_df = pd.read_excel('all_instances_contributions.xlsx')
new_df.shape

# Filter rows where 'True Value' is 1
filtered_df = new_df#[new_df['True Value'] == 1]

# Select columns that end with 'Contribution'
contribution_columns = [col for col in filtered_df.columns if col.endswith('Contribution')]

# Sum contributions for each feature
contributions_sum = filtered_df[contribution_columns].sum().sort_values(ascending=False)

# Get the top 5 features with the highest contribution values
top_features = contributions_sum.head(20)

# Display the top features
print(top_features)

import pandas as pd

def frequency_distribution(df, column_name, true_value_filter=None):
    # Filter the dataframe based on the 'True Value' if a filter is provided
    if true_value_filter is not None:
        df = df[df['True Value'] == true_value_filter]
    
    # Check if the specified column ends with 'Value'
    if column_name.endswith('Value'):
        # Create a frequency distribution for the specified 'Value' column
        freq_distribution = df[column_name].value_counts().to_dict()
        return freq_distribution
    else:
        raise ValueError("The specified column does not end with 'Value'.")

# Example usage:
# df is your dataframe
# column_name is the name of the column you want to analyze
# To get frequency distribution for 'True Value' 1 in the specified column, use:
frequency_distribution(new_df, 'shrm_birth_year Value', 1)

# To get frequency distribution for 'True Value' 0 in the specified column, use:
# frequency_distribution(df, 'YourColumnNameValue', 0)

# To get frequency distribution for both 'True Value' 1 and 0 in the specified column, use:
# frequency_distribution(df, 'YourColumnNameValue')

import pandas as pd
import matplotlib.pyplot as plt

def frequency_distribution(df, column_name, true_value_filter=None):
    # Filter the dataframe based on the 'True Value' if a filter is provided
    if true_value_filter is not None:
        df = df[df['True Value'] == true_value_filter]
    
    # Check if the specified column ends with 'Value'
    if column_name.endswith('Value'):
        # Exclude the 0 value
        df = df[df[column_name] != 0]
        
        # Sort the DataFrame by the 'Year' column
        df = df.sort_values(by='Year')
        
        # Create a frequency distribution for the specified 'Value' column
        freq_distribution = df[column_name].value_counts().sort_index()
        
        # Plot the frequency distribution
        plt.figure(figsize=(10, 6))
        freq_distribution.plot(kind='bar')
        plt.title(f'Frequency Distribution of {column_name} (excluding 0)')
        plt.xlabel('Values')
        plt.ylabel('Frequency')
        plt.show()
        
        return freq_distribution.to_dict()
    else:
        raise ValueError("The specified column does not end with 'Value'.")
    
frequency_distribution(df, 'shrm_birth_year Value', 1)


import pandas as pd
import matplotlib.pyplot as plt

def frequency_distribution(df, column_name, true_value_filter=None):
    # Filter the dataframe based on the 'True Value' if a filter is provided
    if true_value_filter is not None:
        df = df[df['True Value'] == true_value_filter]
    
    # Check if the specified column is 'shrm_birth_year Value'
    if column_name == 'shrm_birth_year Value':
        # Exclude the 0 value
        df = df[df[column_name] != 0]
        
        # Sort the DataFrame by the 'shrm_birth_year Value' column in ascending order
        df = df.sort_values(by=column_name)
        
        # Create a frequency distribution for the 'shrm_birth_year Value' column
        freq_distribution = df[column_name].value_counts().sort_index()
        
        # Plot the frequency distribution
        plt.figure(figsize=(10, 6))
        freq_distribution.plot(kind='bar')
        plt.title(f'Frequency Distribution of {column_name} (excluding 0)')
        plt.xlabel('Years')
        plt.ylabel('Frequency')
        plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
        plt.show()
        
        return freq_distribution.to_dict()
    else:
        raise ValueError(f"The specified column {column_name} is not 'shrm_birth_year Value'.")


frequency_distribution(new_df, 'shrm_birth_year Value', 1)